In [2]:
#!pip install -r requirements.txt

In [3]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

#load environment variables
load_dotenv()


True

In [4]:
chatmodel = ChatOpenAI(model="gpt-3.5-turbo")

In [ ]:
chat_result = chatmodel.invoke("What is special about India?")

In [ ]:
# To get only cotents of the result. It only contains the output message
print("content only: ", chat_result.content)

# To get full message from the chat model
# It contains details of the prompt, model, prompt tokens, result tokens
print("full result: ", chat_result)

# **Examples for basic conversation using langchain and openai model**

In [5]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

# defining messages with its proper type
messages = [
    # setting the chatmodel behaviour for the response of the query
    # It always come first, as it sets the context.
    SystemMessage(content="You are an expert assistant."),
    # setting the real query in message form
    HumanMessage(content="Who won the world series in 2020?")
]

In [ ]:
conv_result = chatmodel.invoke(messages)
print("result message: ", conv_result.content)

In [ ]:
# Now, inorder to keep the conversation going on for longer context window.
PREV_CONTEXT_LIMIT = 10

def get_conversation_result(messages, humanMessage, prevAIMessage):
  if mCount >= PREV_CONTEXT_LIMIT:
    mCount = 0
    print("context limit crossed. Starting conversation with new context")
    while len(messages)>1:
      messages.pop(0)
    messages.append(humanMessage)
    return AIMessage(content=chatmodel.invoke(messages).content), messages
  else:
    messages.append(prevAIMessage)
    messages.append(humanMessage)
    return AIMessage(content=chatmodel.invoke(messages).content), messages



In [ ]:
# Running the loop of conversation for messages

prevAIMessage = AIMessage(content=chatmodel.invoke(messages).content)

while True:
  query = input(" User: ")
  if query.lower() == "exit":
    break
  # get human message from user query only for each coversation loop as a form of input
  humanMessage = HumanMessage(content=query)

  # get chat result as well as updated messages
  prevAIMessage, messages = get_conversation_result(messages, humanMessage, prevAIMessage)



# **Understanding the chat Prompt template for building chatbots**

In [6]:
# ChatPrompt template helps in creating custom chat query with parameters.
# It can be seen as masked prompt template.
from langchain.prompts import ChatPromptTemplate

chatTemplate = "Explain this {topic} in details"
prompt_template = ChatPromptTemplate.from_template(chatTemplate)



In [ ]:
prompt = prompt_template.invoke({"topic": "Machine Learning"})

chatResult = chatmodel.invoke([prompt])
print(chatResult.content)

In [7]:
# We can also change the context of the messages on runtime(dynamically)
# Like changing the context of the SystemAI message

advancedTemplate = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert in the {topic}."),
        ("human", "Please explain me in details about the {subject} along with its pros & cons.")
    ]
)



In [ ]:
advancedPrompt = advancedTemplate.invoke({"topic": "Machine Learning", "subject": "Decision Trees"})

advancedResult = chatmodel.invoke(advancedPrompt)
print(advancedResult.content)

# **Chaining of the langchain components**

Since, each langchain components are Runnable instances, where we can call invoke() calls. Thus, we can compose these runnables using (|) pipe operator. The output of previous component will be taken as input for next component in a sequential manner.

This property is achieved due to LangChain Expression Language (LCEL).


In [8]:
# creating chain with prompt and chat model

chain1 = prompt_template | chatmodel

chain2 = advancedTemplate | chatmodel


In [ ]:
# executing chains

resut1 = chain1.invoke({"topic": "Machine Learning and AI"})

result2 = chain2.invoke({"topic": "Machine Learning and AI", "subject": "Gaussian Splatting"})

In [ ]:
# The output from above is in langshain component - AIMessage(.......)
# In order to get raw string output, we can use String OutputParser component of
# langchain.

from langchain_core.output_parsers import StrOutputParser

string_result1 = chain1 | StrOutputParser()
string_result2 = chain2 | StrOutputParser()

# Note: above statement is like: prompt_template | chatmodel | StrOutputParser()


# Now, we can execute it using invoke function

output1 = string_result1.invoke({"topic": "Machine Learning and AI"})

output2 = string_result2.invoke({"topic": "Machine Learning and AI", "subject": "Gaussian Splatting"})

print("\n result 1:  ", output1)
print("\n result 2:  ", output2)

In [10]:
# Creating more complex and parallel branched version of langChain

from langchain_core.runnables import RunnableParallel, RunnableLambda

branch1 = (
    RunnableLambda(lambda x: x.upper()) | StrOutputParser()
)
branch2 = (
    RunnableLambda(lambda x: x.lower()) | StrOutputParser()
)

full_flow = (
    advancedTemplate
    | chatmodel
    | StrOutputParser()
    | RunnableParallel(branches={"upper": branch1, "lower": branch2})
    | RunnableLambda(lambda x: print(" output value: ", x["upper"] + x["lower"]))
)


In [ ]:
# Executing full flow

full_flow_result = full_flow.invoke({"topic": "Machine Learning and AI", "subject": "Gaussian Splatting"})

print(full_flow_result)

In [ ]:
# Understanding RunnableBranch for Conditional branches
# Taking an example of topic explanation based on user query

mathematical_explanation = ChatPromptTemplate.from_messages(
    [
        ("system", "You are expert in AI concepts"),
    ("human", "Generate the explanation about given {topic} in mathematical way only"),
    ]
)

theoretical_explanation = ChatPromptTemplate.from_messages(
    [
        ("system", "You are expert in AI concepts"),
    ("human", "Generate the explanation about given {topic} in theoretical way only"),
    ]
)

combined_explanation = ChatPromptTemplate.from_messages(
    [
        ("system", "You are expert in AI concepts"),
    ("human", "Generate the explanation about given {topic} in both mathematical and theoretical way"),
    ]
)

In [ ]:
# Creating branches with condition
from langchain_core.runnables import RunnableBranch
branches_values = RunnableBranch(
    (lambda x: "mathematical" in x["topic"].lower(), mathematical_explanation | chatmodel | StrOutputParser()),
    (lambda x: "theoretical" in x["topic"].lower(), theoretical_explanation | chatmodel | StrOutputParser()),
    (lambda x: True, combined_explanation | chatmodel | StrOutputParser())
)

branch_flow_result = branches_values.invoke({"topic": "Machine Learning and AI with mathematical description"})

print(branch_flow_result)